In [1]:
# basics
import numpy as np
import pandas as pd
import datetime
import re
import os,glob
import pickle
import textwrap

# statistics
import statsmodels.api as sm
import statsmodels.formula.api as smf


# plotting
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [15,12]

# own data wrappers
from imp import reload
import measureclass as mc; reload(mc);
import coronadataclass as cdc; reload(cdc);
import crossvalidationclass as cvc; reload(cvc);

np.seterr(divide = 'ignore');


In [2]:
cccsl     = mc.COVID19_measures(datasource = 'CSH',       download_data = True)
oxford    = mc.COVID19_measures(datasource = 'OXFORD',    download_data = True)
whophsm   = mc.COVID19_measures(datasource = 'WHOPHSM',   download_data = True)
acaps     = mc.COVID19_measures(datasource = 'ACAPS',     download_data = True)
coronanet = mc.COVID19_measures(datasource = 'CORONANET', download_data = True)
hitcovid  = mc.COVID19_measures(datasource = 'HITCOVID',  download_data = True, dateformat = '%Y-%m-%d')

In [5]:
print('           {:8s} {:9s}'.format('Measures','Countries'))
print('CSH:       {:8d} {:9d}'.format(cccsl.MeasureList(mincount = 5).shape[0],     len(cccsl.countrylist)))
print('OXFORD:    {:8d} {:9d}'.format(oxford.MeasureList(mincount = 5).shape[0],    len(oxford.countrylist)))
print('WHOPHSM:   {:8d} {:9d}'.format(whophsm.MeasureList(mincount = 5).shape[0],   len(whophsm.countrylist)))
print('ACAPS:     {:8d} {:9d}'.format(acaps.MeasureList(mincount = 5).shape[0],     len(acaps.countrylist)))
print('CORONANET: {:8d} {:9d}'.format(coronanet.MeasureList(mincount = 5).shape[0], len(coronanet.countrylist)))
print('HITCOVID:  {:8d} {:9d}'.format(hitcovid.MeasureList(mincount = 5).shape[0],  len(hitcovid.countrylist)))


           Measures Countries
CSH:             55        57
OXFORD:          17       178
WHOPHSM:         41       213
ACAPS:           34       194
CORONANET:      109       198
HITCOVID:        39       136


In [7]:
cccsl.ImplementationTable(country = 'Austria', mincount= 5)

,ActivateCaseNotification,ActivateOrEstablishEmergencyResponse,ActivelyCommunicateWithManagers,AdaptProceduresForPatientManagement,AirportHealthCheck,AirportRestriction,BorderHealthCheck,BorderRestriction,ClosureOfEducationalInstitutions,CordonSanitaire,...,MeasuresToEnsureSecurityOfSupply,NationalLockdown,PersonalProtectiveMeasures,PublicTransportRestriction,Quarantine,ReOpeningOfEducationalInstitutions,ReturnOperationOfNationals,SmallGatheringCancellation,TracingAndTracking,TravelAlertAndWarning
22/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30/07/2020,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
31/07/2020,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
01/08/2020,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
02/08/2020,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
